 # MTH8408 : Méthodes d'optimisation et contrôle optimal
 ## Laboratoire 2: Optimisation sans contraintes
Tangi Migot

In [2]:
using Pkg;
Pkg.add("ADNLPModels");
Pkg.add("LinearAlgebra");
Pkg.add("NLPModels");
Pkg.add("Printf");

    Updating registry at `C:\Users\Goglu\.julia\registries\General.toml`


`ADNLPModels` is pinned at `v0.3.1`: maintaining pinned version
   Resolving package versions...

  No Changes to `C:\Users\Goglu\.julia\environments\v1.8\Project.toml`
  No Changes to `C:\Users\Goglu\.julia\environments\v1.8\Manifest.toml`


   Resolving package versions...

  No Changes to `C:\Users\Goglu\.julia\environments\v1.8\Project.toml`
  No Changes to `C:\Users\Goglu\.julia\environments\v1.8\Manifest.toml`


   Resolving package versions...

    Updating `C:\Users\Goglu\.julia\environments\v1.8\Project.toml`
⌅ [a4795742] + NLPModels v0.18.5
  No Changes to `C:\Users\Goglu\.julia\environments\v1.8\Manifest.toml`


   Resolving package versions...

    Updating `C:\Users\Goglu\.julia\environments\v1.8\Project.toml`
  [de0858da] + Printf
  No Changes to `C:\Users\Goglu\.julia\environments\v1.8\Manifest.toml`

In [3]:
using ADNLPModels, LinearAlgebra, NLPModels, Printf

On pourra trouver de la documentation sur `ADNLPModels` et `NLPModels` ici:
- [juliasmoothoptimizers.github.io/NLPModels.jl/dev/](https://juliasmoothoptimizers.github.io/NLPModels.jl/dev/)
- [juliasmoothoptimizers.github.io/ADNLPModels.jl/dev/](https://juliasmoothoptimizers.github.io/ADNLPModels.jl/dev/)

In [4]:
# Problème test:
f(x) = x[1]^2 * (2*x[1] - 3) - 6*x[1]*x[2] * (x[1] - x[2] - 1) # fonction objectif vue en classe
g(x) = 6 * [x[1]^2 - x[1] - 2*x[1]*x[2] + x[2]^2 + x[2]; -x[1]^2 + 2*x[1]*x[2] + x[1]] # le gradient de f
H(x) = 6 * [2*x[1]-1-2*x[2] -2*x[1]+2*x[2]+1; -2*x[1]+2*x[2]+1 2*x[1]] # la Hessienne de f

H (generic function with 1 method)

### Exercice 1: Newton avec recherche linéaire - amélioration du code

Ci-dessous, vous avez le code de deux fonctions qui ont été vues dans le cours, la recherche linéaire qui satisfait Armijo, et une méthode de Newton avec cette recherche linéaire. Le but de ce laboratoire est d'implémenter d'autres méthodes utiles pour résoudre des problèmes de grandes dimensions.

In [5]:
#Amélioration possibles: return also the value of f
function armijo(xk, dk, fk, gk, f)
  slope = dot(gk, dk) #doit être <0
  t = 1.0
  while f(xk + t * dk) > fk + 1.0e-4 * t * slope
    t /= 1.5
  end
  return t
end

armijo (generic function with 1 method)

In [6]:
#Test pour vérifier que la fonction armijo fonctionne correctement.
using Test #le package Test définit (entre autre) la macro @test qui permet de faire des tests unitaires :-)
xk = ones(2)
gk = g(xk)
dk = - gk
fk = f(xk)
t  = armijo(xk, dk, fk, gk, f)
@test t < 1
@test f(xk + t * dk) <= fk + 1.0e-4 * t * dot(gk,dk)

xk = [1.5, 0.5]
fk = f(xk)
gk = g(xk)
dk = - gk
t  = armijo(xk, dk, fk, gk, f)
@test t < 1
@test f(xk + t * dk) <= f(xk) + 1.0e-4 * t * dot(g(xk),dk)


Test Passed

In [7]:
function newton_armijo(f, g, H, x0; verbose::Bool = true)
  xk  = x0
  fk  = f(xk)
  gk = g(xk)
  gnorm = gnorm0 = norm(gk)
  k = 0
  verbose && @printf "%2s %9s %9s\n" "k" "fk" "||∇f(x)||"
  verbose && @printf "%2d %9.2e %9.1e\n" k fk gnorm
  while gnorm > 1.0e-6 + 1.0e-6 * gnorm0 && k < 100
    Hk = H(xk)
    dk = - Hk \ gk
    slope = dot(dk, gk)
    λ = 0.0
    while slope ≥ -1.0e-4 * norm(dk) * gnorm
      λ = max(1.0e-3, 10 * λ)
      dk = - ((Hk + λ * I ) \ gk)
      slope = dot(dk, gk)
    end
    t = armijo(xk, dk, fk, gk, f)
    xk += t * dk
    fk = f(xk)
    gk = g(xk)
    gnorm = norm(gk)
    k += 1
    verbose && @printf "%2d %9.2e %9.1e %7.1e \n" k fk gnorm t
  end
  return xk
end

newton_armijo (generic function with 1 method)

In [8]:
sol = newton_armijo(f, g, H, [.5, .5])
@test g(sol) ≈ zeros(2) atol = 1.0e-6

 k        fk ||∇f(x)||


 0  1.00e+00   4.5e+00
 1  5.08e-02   8.4e-01 1.0e+00 
 2  4.73e-04   7.6e-02 1.0e+00 
 3  6.97e-08   9.1e-04 1.0e+00 
 4  1.62e-15   1.4e-07 1.0e+00 


Test Passed

On veut améliorer le code de la fonction `newton_armijo` avec les ajouts suivants:
- Changer les paramètre d'entrées de la fonction pour un `nlp`
- Avant d'appeler la recherche linéaire, si `slope = dot(dk, gk)` est plus grand que `-1.0e-4 * norm(dk) * gnorm`, on modifie le système. On fait maximum 5 mise à jour de `λ`, sinon on prend l'opposé du gradient.
```
    λ = 0.0
    while slope ≥ -1.0e-4 * norm(dk) * gnorm
      λ = max(1.0e-3, 10 * λ)
      dk = - ((Hk + λ * I ) \ gk)
      slope = dot(dk, gk)
    end
```
Ajouter un compteur sur le nombre de mises à jour de `λ` et ajuster `dk = - gk` si la limite est atteinte.
- On veut aussi détecter et éventuellement arrêter la boucle `while` si la fonction objectif `fk` devient trop petite/négative (inférieure à `-1e15`), i.e. le problème est non-bornée inférieurement.
- On veut ajouter deux critères d'arrêts supplémentaires: 
  - un compteur sur le nombre d'évaluations de f (maximum 1000). Utiliser `neval_obj(nlp)`.
  -  une limite de temps d'execution, `max_time = 60.0`. Utiliser la fonction `time()`.
- Enfin, on voudrait aussi voir un message à l'écran si l'algorithme n'a pas trouvé la solution, i.e. il s'est arrêté à cause de la limite sur le nombre d'itérations, temps, évaluation de fonctions, problème non-borné.

In [16]:
#SOLUTION: fonction à modifier
function newton_armijo(nlp, x0; verbose::Bool = true)
  start_timer = time()
  xk = x0
  fk = obj(nlp, xk)
  gk = grad(nlp, xk)
  gnorm = gnorm0 = norm(gk)
  k = 0
  verbose && @printf "%2s %9s %9s\n" "k" "fk" "||∇f(x)||"
  verbose && @printf "%2d %9.2e %9.1e\n" k fk gnorm
  continue_eval = true
  while continue_eval
    Hk = hess(nlp, xk)
    dk = - Hk \ gk
    slope = dot(dk, gk)
    λ = 0.0
    λ_counter = 0
    while slope ≥ -1.0e-4 * norm(dk) * gnorm && λ_counter < 5
      λ = max(1.0e-3, 10 * λ)
      dk = - ((Hk + λ * I ) \ gk)
      slope = dot(dk, gk)
      λ_counter += 1
      if λ_counter == 5 && slope ≥ -1.0e-4 * norm(dk) * gnorm
        dk = -gk
      end
    end
    t = armijo(xk, dk, fk, gk, x->obj(nlp, x))
    xk += t * dk
    fk = obj(nlp, xk)
    gk = grad(nlp, xk)
    gnorm = norm(gk)
    k += 1
    verbose && @printf "%2d %9.2e %9.1e %7.1e \n" k fk gnorm t
    if gnorm <= 1.0e-6 + 1.0e-6 * gnorm0
      # Solution found
      continue_eval = false
    elseif neval_obj(nlp) > 1000
      continue_eval = false
      @printf "\nMETHOD FAILED - Reached Maximum Number of Evaluations (1000) of Objective Function \n"
    elseif time() - start_timer > 60.0
      continue_eval = false
      @printf "\nMETHOD FAILED - Reached Maximum Time of Method (60.0s) \n"
    elseif fk < -1e15
      continue_eval = false
      @printf "\nMETHOD FAILED - Objective Function is not Bounded (fk < -1e15) \n"
    elseif k >= 100
      continue_eval = false
      @printf "\nMETHOD FAILED - Reached Maximum Number of Iterations (100) \n"
    end
  end
  return xk
end

newton_armijo (generic function with 2 methods)

In [19]:
#Test
f(x) = x[1]^2 * (2*x[1] - 3) - 6*x[1]*x[2] * (x[1] - x[2] - 1)
x0 = ones(2)
nlp = ADNLPModel(f, x0)

newton_armijo(nlp, x0)

 k        fk ||∇f(x)||
 0  5.00e+00   1.2e+01
 1  4.07e-01   2.7e+00 1.0e+00 
 2  1.39e-02   4.3e-01 1.0e+00 
 3  4.63e-05   2.4e-02 1.0e+00 
 4  6.99e-10   9.2e-05 1.0e+00 
 5  1.63e-19   1.4e-09 1.0e+00 


2-element Vector{Float64}:
 2.3283064370807974e-10
 2.3283064370807974e-10

In [20]:
# Tests secrets 1: contactez votre professeur pour avoir ces tests additionels si vous êtes prêt.
using Test

@testset "Test set for newton_armijo" begin
    #Test problem:
    fH(x) = (x[2]+x[1].^2-11).^2+(x[1]+x[2].^2-7).^2
    x0H = [10., 20.]
    himmelblau = ADNLPModel(fH, x0H)

    problem2 = ADNLPModel(x->-x[1]^3 + x[2]^2 + x[3]^2, ones(3))

    roz(x) = 100 *  (x[2] - x[1]^2)^2 + (x[1] - 1.0)^2
    rosenbrock = ADNLPModel(roz, [-1.2, 1.0])

    f(x) = x[1]^2 * (2*x[1] - 3) - 6*x[1]*x[2] * (x[1] - x[2] - 1)
    pb_du_cours = ADNLPModel(f, [-1.001, -1.001]) #ou [1.5, .5] ou [.5, .5]


    ######################################### newton_armijo ##################

    #Unit/Validation Tests
    ep1 = 1e-6 + norm(grad(himmelblau, himmelblau.meta.x0)) * 1e-6
    sol = newton_armijo(himmelblau, himmelblau.meta.x0, verbose = false)
    @test norm(grad(himmelblau, sol)) ≤ ep1
    @test sol ≈ [3, 2] atol = ep1

    ep2 = 1e-6 + norm(grad(problem2, problem2.meta.x0)) * 1e-6
    sol =  newton_armijo(problem2, problem2.meta.x0, verbose = false)
    # @test stats.status == :unbounded
    @test obj(problem2, sol) ≤ -1e15

    ep2 = 1e-6 + norm(grad(rosenbrock, rosenbrock.meta.x0)) * 1e-6
    sol = newton_armijo(rosenbrock, rosenbrock.meta.x0, verbose = false)
    @test sol ≈ [1., 1.] atol = ep2

    ep3 = 1e-6 + norm(grad(pb_du_cours, [-1.001, -1.001])) * 1e-6
    sol = newton_armijo(pb_du_cours, [-1.001, -1.001], verbose = false)
    @test norm(grad(pb_du_cours, sol)) ≤ ep3 || obj(pb_du_cours, sol) <= -1e15

    ep4 = 1e-6 + norm(grad(pb_du_cours, [1.5, .5])) * 1e-6
    sol = newton_armijo(pb_du_cours, [1.5, .5], verbose = false)
    @test norm(grad(pb_du_cours, sol)) ≤ ep4 || obj(pb_du_cours, sol) <= -1e15

    ep5 = 1e-6 + norm(grad(pb_du_cours, [.5, .5])) * 1e-6
    sol = newton_armijo(pb_du_cours, [.5, .5], verbose = false)
    @test norm(grad(pb_du_cours, sol)) ≤ ep5 || obj(pb_du_cours, sol) <= -1e15
end



METHOD FAILED - Objective Function is not Bounded (fk < -1e15) 



METHOD FAILED - Objective Function is not Bounded (fk < -1e15) 


Test Summary:              | Pass  Total  Time
Test set for newton_armijo |    7      7  4.8s


Test.DefaultTestSet("Test set for newton_armijo", Any[], 7, false, false, true, 1.674490559776e9, 1.674490564557e9)

### Exercice 2: LDLt-Newton avec recherche linéaire

On va maintenant modifier la méthode de Newton vu précédemment pour utiliser un package qui s'occupe de calculer une factorisation de la matrice hessienne tel que:
$$
\nabla^2 f(x) = LDL^T.
$$
Ce type de factorisation n'est possible que si la matrice hessienne est définie positive, dans le cas contraire on a besoin de régularisé le système comme dans l'exercice précédent.

Pour résoudre le système linéaire en utilisant cette factorisation, on va utiliser le package [`LDLFactorizations`](https://github.com/JuliaSmoothOptimizers/LDLFactorizations.jl):

In [22]:
using Pkg;
Pkg.add("LDLFactorizations");

   Resolving package versions...


    Updating `C:\Users\Goglu\.julia\environments\v1.8\Project.toml`
  [40e66cde] + LDLFactorizations v0.10.0
  No Changes to `C:\Users\Goglu\.julia\environments\v1.8\Manifest.toml`

In [23]:
using LDLFactorizations, LinearAlgebra

Un tutoriel sur l'utilisation de `LDLFactorizations` est disponible sur la documentation du package sur github ou encore [à ce lien](https://juliasmoothoptimizers.github.io/LDLFactorizations.jl/dev/tutorial/).

Voici un exemple d'utilisation de ce package. La matrice dont on veut calculer la factorisation doit être de type `Symmetric`.

In [26]:
A = ones(2,2) #cette matrice symétrique, mais pas du type Symmetric
              #à noter que cette matrice n'est pas définie positive.
typeof(A) <: Symmetric #false
A = Symmetric(A)
typeof(A) <: Symmetric #true :)

true

Deuxième étape, le package fait une phase d'analyse de la matrice avec `ldl_analyze` en créant une structure pratique pour les diverses fonctions du package.

In [27]:
A = -rand(2, 2)
sol = rand(2)
b = A*sol #on veut résoudre le système A*x=b

# LDLFactorizations va en réalité demander la matrice triangulaire supérieure
A = Symmetric(triu(A), :U)
S = ldl_analyze(A)
ldl_factorize!(A, S)
x = S \ b # x = A \b ça va être résolu par Julia 
norm(A * x - b)

5.003707553108401e-17

In [28]:
A = [0. 1.; 1. 0.]

2×2 Matrix{Float64}:
 0.0  1.0
 1.0  0.0

In [29]:
A = Symmetric(triu(A), :U)
S = ldl_analyze(A)
ldl_factorize!(A, S)

LDLFactorizations.LDLFactorization{Float64, Int64, Int64, Int64}(true, false, true, 2, [2, -1], [0, 0], [1, 2], [1, 2], [1, 2], [1, 2, 2], [1, 1, 1], Int64[], [140720410276112], [8.867834394367e-312], [0.0, 8.867598568297e-312], [0.0, 8.848722448325e-312], [1795186567736, 1795186567776], 0.0, 0.0, 0.0, 2)

In [30]:
S.d

2-element Vector{Float64}:
 0.0
 8.867598568297e-312

La matrice `A` factorisée par $LDL^T$ n'était pas forcément définie positive. On peut le voir sur les valeurs de $D$.

In [31]:
S.d #c'est le vecteur qui correspond à la matrice diagonale D.

2-element Vector{Float64}:
 0.0
 8.867598568297e-312

Pour l'optimisation, dans le cas où des valeurs de $D$ sont négatives, i.e. `minimum(S.d) <= 0.`, on ajoutera une correction pour être sûr d'obtenir une direction de descente. On pourra choisir un des deux:
- `S.d   = abs.(S.d)`
- `S.d .+= -minimum(S.d) + 1e-6`

#### Utiliser cette technique pour calculer la direction de descente:

In [54]:
# Solution: modifier le calcul de la direction avec LDLFactorizations
function newton_ldlt_armijo(nlp, x0; verbose::Bool = true)
  xk  = x0
  fk  = obj(nlp, xk)
  gk = grad(nlp, xk)
  gnorm = gnorm0 = norm(gk)
  k = 0
  verbose && @printf "%2s %9s %9s\n" "k" "fk" "||∇f(x)||"
  verbose && @printf "%2d %9.2e %9.1e\n" k fk gnorm
  while gnorm > 1.0e-6 + 1.0e-6 * gnorm0 && k < 100 && fk > -1e15
    Hk = Symmetric(triu(hess(nlp, xk)), :U)
    Sk = ldl_analyze(Hk)
    ldl_factorize!(Hk, Sk)
    if minimum(Sk.d) <= 0.
      Sk.d .+= -minimum(Sk.d) + 1e-6
    end
    dk = - Sk \ gk
    slope = dot(dk, gk)
    t = armijo(xk, dk, fk, gk, x -> obj(nlp, x))
    xk += t * dk
    fk = obj(nlp, xk)
    gk = grad(nlp, xk)
    gnorm = norm(gk)
    k += 1
    verbose && @printf "%2d %9.2e %9.1e %7.1e \n" k fk gnorm t
  end
  return xk
end

newton_ldlt_armijo (generic function with 1 method)

In [55]:
#Test
f(x) = x[1]^2 * (2*x[1] - 3) - 6*x[1]*x[2] * (x[1] - x[2] - 1)
nlp = ADNLPModel(f, zeros(2))

newton_ldlt_armijo(nlp, x0)

 k        fk ||∇f(x)||
 0  5.00e+00   1.2e+01
 1  1.00e+00   4.5e+00 1.0e+00 
 2  2.19e-01   1.9e+00 1.0e+00 
 3  5.08e-02   8.4e-01 1.0e+00 
 4  1.22e-02   4.0e-01 1.0e+00 
 5  2.99e-03   2.0e-01 1.0e+00 
 6 -9.82e-01   5.0e-01 1.3e-04 
 7 -1.00e+00   3.1e-02 1.0e+00 
 8 -1.00e+00   1.4e-04 1.0e+00 
 9 -1.00e+00   2.4e-09 1.0e+00 


2-element Vector{Float64}:
 1.0000000002802167
 7.107141154069226e-13

In [56]:
# Tests secrets 2: contactez votre professeur pour avoir ces tests additionels si vous êtes prêt.
using Test

@testset "Test set for newton_ldlt_armijo" begin
    #Test problem:
    fH(x) = (x[2]+x[1].^2-11).^2+(x[1]+x[2].^2-7).^2
    x0H = [10., 20.]
    himmelblau = ADNLPModel(fH, x0H)

    problem2 = ADNLPModel(x->-x[1]^3 + x[2]^2 + x[3]^2, ones(3))

    roz(x) = 100 *  (x[2] - x[1]^2)^2 + (x[1] - 1.0)^2
    rosenbrock = ADNLPModel(roz, [-1.2, 1.0])

    f(x) = x[1]^2 * (2*x[1] - 3) - 6*x[1]*x[2] * (x[1] - x[2] - 1)
    pb_du_cours = ADNLPModel(f, [-1.001, -1.001]) #ou [1.5, .5] ou [.5, .5]

    #Unit/Validation Tests
    ep1 = 1e-6 + norm(grad(himmelblau, himmelblau.meta.x0)) * 1e-6
    sol = newton_ldlt_armijo(himmelblau, himmelblau.meta.x0, verbose = false)
    @test norm(grad(himmelblau, sol)) ≤ ep1
    @test sol ≈ [3, 2] atol = ep1

    ep2 = 1e-6 + norm(grad(problem2, problem2.meta.x0)) * 1e-6
    sol = newton_ldlt_armijo(problem2, problem2.meta.x0, verbose = false)
    # @test stats.status == :unbounded
    @test obj(problem2, sol) ≤ -1e15

    ep2 = 1e-6 + norm(grad(rosenbrock, rosenbrock.meta.x0)) * 1e-6
    sol = newton_ldlt_armijo(rosenbrock, rosenbrock.meta.x0, verbose = false)
    @test sol ≈ [1., 1.] atol = ep2

    ep3 = 1e-6 + norm(grad(pb_du_cours, [-1.001, -1.001])) * 1e-6
    sol = newton_ldlt_armijo(pb_du_cours, [-1.001, -1.001], verbose = false)
    @test norm(grad(pb_du_cours, sol)) ≤ ep3 || obj(pb_du_cours, sol) <= -1e15

    ep4 = 1e-6 + norm(grad(pb_du_cours, [1.5, .5])) * 1e-6
    sol = newton_ldlt_armijo(pb_du_cours, [1.5, .5], verbose = false)
    @test norm(grad(pb_du_cours, sol)) ≤ ep4 || obj(pb_du_cours, sol) <= -1e15

    ep5 = 1e-6 + norm(grad(pb_du_cours, [.5, .5])) * 1e-6
    sol = newton_ldlt_armijo(pb_du_cours, [.5, .5], verbose = false)
    @test norm(grad(pb_du_cours, sol)) ≤ ep5 || obj(pb_du_cours, sol) <= -1e15
end


Test Summary:                   | Pass  Total  Time
Test set for newton_ldlt_armijo |    7      7  2.1s


Test.DefaultTestSet("Test set for newton_ldlt_armijo", Any[], 7, false, false, true, 1.674491920911e9, 1.674491923045e9)

### Exercice 3: Méthode quasi-Newton: BFGS

#### Méthode quasi-Newton: BFGS
Pour des problèmes de très grandes tailles, il est parfois très coûteux d'évaluer la hessienne du problème d'optimisation (et même le produit hessienne-vecteur). La famille des méthode *quasi-Newton* construit une approximation $B_k$ symétrique définie positive de la matrice Hessienne en utilisant seulement le gradient et en mesurant sa variation, et permet quand même d'améliorer significativement les performances comparé à la méthode du gradient.
$$
s_k = x_{k+1} - x_k, \quad y_k = \nabla f(x_{k+1}) - \nabla f(x_k).
$$
Par ailleurs la matrice $B_k$ est aussi construite de façon à ce que l'inverse soit connue, il n'y a donc pas de système linéaire à résoudre.

La méthode la plus connue dans la famille des méthodes quasi-Newton, est la méthode BFGS (Broyden - Fletcher, Goldfarb, and Shanno), et utilise la formule suivante pour calculer l'inverse de $B_k$ que l'on note $H_k$:
$$
H_{k+1} = (I - \rho_k s_ky_k^T)H_k(I-\rho_ky_ks_k^T) + \rho_ks_ks_k^T, \quad \rho_k = \frac{1}{y_k^Ts_k}.
$$
L'algorithme est presque le même que la méthode de Newton à la différence qu'il n'y a pas de système linéaire à résoudre et la direction $d_k$ est à coup sûr une direction de descente. Ainsi la direction de descente est calculée comme suit:
$$
d_k = - H_k \nabla f(x_k).
$$

Comment choisir la matrice $H_0$? On peut éventuellement choisir $I$. Une alternative est d'utiliser $H_0=I$ pour la première itération et ensuite mettre $H_0$ à jour avant de calculer $H_1$ en utilisant:
$$
H_0 = \frac{y_k^Ts_k}{y_k^Ty_k}I.
$$

**Important**: pour s'assurer que la matrice $H_k$ reste définie positive à toutes les itérations, il faut s'assurer que $y_k^Ts_k>0$. C'est toujours vrai pour des fonctions convexes, mais pas nécessairement dans le cas général. On pourra tester ici la version "skip" qui ne mets pas à jour quand cette condition n'est pas vérifiée.

In [117]:
# Solution: copier-coller votre newton_armijo ici et modifier le calcul de la direction avec la méthode de BFGS inverse skip.
function bfgs_quasi_newton_armijo(nlp, x0; verbose::Bool = true)
  start_timer = time()
  xk = x0
  fk = obj(nlp, xk)
  gk = grad(nlp, xk)
  gnorm = gnorm0 = norm(gk)
  k = 0
  verbose && @printf "%2s %9s %9s\n" "k" "fk" "||∇f(x)||"
  verbose && @printf "%2d %9.2e %9.1e\n" k fk gnorm
  continue_eval = true
  H0 = I
  sk = xk
  yk = gk
  Hk = H0
  while continue_eval
    if k == 0
      Hk = H0
    elseif yk'*sk > 0
      Hk = (k==1 ? I *(yk'*sk) / (yk'*yk) : Hk)
      pk = 1 / (yk'*sk)
      Hk = (I - pk*sk*yk') * Hk * (I - pk*yk*sk') + pk*sk*sk'
    end
    dk = -Hk * gk
    t = armijo(xk, dk, fk, gk, x->obj(nlp, x))
    sk = t * dk
    yk = grad(nlp, xk + sk) - gk
    xk += sk
    fk = obj(nlp, xk)
    gk = grad(nlp, xk)
    gnorm = norm(gk)
    k += 1
    verbose && @printf "%2d %9.2e %9.1e %7.1e \n" k fk gnorm t
    if gnorm <= 1.0e-6 + 1.0e-6 * gnorm0
      # Solution found
      continue_eval = false
    elseif neval_obj(nlp) > 1000
      continue_eval = false
      @printf "\nMETHOD FAILED - Reached Maximum Number of Evaluations (1000) of Objective Function \n"
    elseif time() - start_timer > 60.0
      continue_eval = false
      @printf "\nMETHOD FAILED - Reached Maximum Time of Method (60.0s) \n"
    elseif fk < -1e15
      continue_eval = false
      @printf "\nMETHOD FAILED - Objective Function is not Bounded (fk < -1e15) \n"
    elseif k >= 100
      continue_eval = false
      @printf "\nMETHOD FAILED - Reached Maximum Number of Iterations (100) \n"
    end
  end
  return xk
end

bfgs_quasi_newton_armijo (generic function with 1 method)

In [75]:
#Test
f(x) = x[1]^2 * (2*x[1] - 3) - 6*x[1]*x[2] * (x[1] - x[2] - 1)
nlp = ADNLPModel(f, zeros(2))
x0 = ones(2)*2

bfgs_quasi_newton_armijo(nlp, x0)

 k        fk ||∇f(x)||
 0  2.80e+01   3.6e+01
 1  5.42e+00   2.0e+01 5.9e-02 
 2 -2.46e-01   2.3e+00 1.0e+00 
 3 -4.87e-01   1.8e+00 1.0e+00 
 4 -9.56e-01   6.7e-01 1.0e+00 
 5 -9.96e-01   3.0e-01 1.0e+00 
 6 -1.00e+00   1.2e-01 1.0e+00 
 7 -1.00e+00   4.4e-02 1.0e+00 
 8 -1.00e+00   4.5e-03 1.0e+00 
 9 -1.00e+00   7.4e-04 1.0e+00 
10 -1.00e+00   8.5e-05 1.0e+00 
11 -1.00e+00   3.1e-06 1.0e+00 


2-element Vector{Float64}:
 1.0000000568101004
 2.64986337366262e-7

In [120]:
# Tests secrets 3: contactez votre professeur pour avoir ces tests additionels si vous êtes prêt.
using Test

@testset "Test set for bfgs_quasi_newton_armijo" begin
	#Test problem:
	fH(x) = (x[2]+x[1].^2-11).^2+(x[1]+x[2].^2-7).^2
	x0H = [10., 20.]
	himmelblau = ADNLPModel(fH, x0H)

	problem2 = ADNLPModel(x->-x[1]^3 + x[2]^2 + x[3]^2, ones(3))

	roz(x) = 100 *  (x[2] - x[1]^2)^2 + (x[1] - 1.0)^2
	rosenbrock = ADNLPModel(roz, [-1.2, 1.0])

	f(x) = x[1]^2 * (2*x[1] - 3) - 6*x[1]*x[2] * (x[1] - x[2] - 1)
	pb_du_cours = ADNLPModel(f, [-1.001, -1.001]) #ou [1.5, .5] ou [.5, .5]


	######################################### bfgs_quasi_newton_armijo ##################

	#Unit/Validation Tests
	using Logging, Test
    ep1 = 1e-6 + norm(grad(himmelblau, himmelblau.meta.x0)) * 1e-6
	sol = bfgs_quasi_newton_armijo(himmelblau, himmelblau.meta.x0) 
	@test norm(grad(himmelblau, sol)) ≤ ep1

    ep4 = 1e-6 + norm(grad(pb_du_cours, [1.5, .5])) * 1e-6
	sol = bfgs_quasi_newton_armijo(pb_du_cours, [1.5, .5]) 
	@test norm(grad(pb_du_cours, sol)) ≤ ep4
	@test sol ≈ [1, 0] atol = ep4
	@show sol

    ep5 = 1e-6 + norm(grad(pb_du_cours, [.5, .5])) * 1e-6
	sol = bfgs_quasi_newton_armijo(pb_du_cours, [.5, .5]) 
	@test norm(grad(pb_du_cours, sol)) ≤ ep4
	@test sol ≈ [1, 0] atol = ep4
	@show sol
end

 k        fk ||∇f(x)||
 0  1.74e+05   3.3e+04
 1  2.73e+04   8.6e+03 1.0e-03 
 2  1.26e+03   8.6e+02 1.0e+00 
 3  7.23e+02   5.8e+02 1.0e+00 
 4  1.70e+02   2.1e+02 1.0e+00 
 5  5.18e+01   9.6e+01 1.0e+00 
 6  1.16e+01   4.0e+01 1.0e+00 
 7  2.05e+00   1.7e+01 1.0e+00 
 8  2.54e-01   6.9e+00 1.0e+00 
 9  7.20e-02   3.8e+00 1.0e+00 
10  5.50e-02   3.0e+00 1.0e+00 
11  3.86e-02   2.2e+00 1.0e+00 
12  1.55e-02   1.0e+00 1.0e+00 
13  2.85e-03   4.4e-01 1.0e+00 
14  1.71e-04   1.7e-01 1.0e+00 


15  3.81e-06   2.8e-02 1.0e+00 
 k        fk ||∇f(x)||
 0  0.00e+00   4.5e+00
 1 -3.73e-01   4.2e+00 8.8e-02 
 2 -7.22e-01   1.4e+00 1.0e+00 
 3 -8.44e-01   9.3e-01 1.0e+00 
 4 -9.83e-01   4.6e-01 1.0e+00 
 5 -9.98e-01   2.3e-01 1.0e+00 
 6 -1.00e+00   7.9e-02 1.0e+00 
 7 -1.00e+00   1.3e-02 1.0e+00 
 8 -1.00e+00   9.1e-04 1.0e+00 
 9 -1.00e+00   1.3e-04 1.0e+00 
10 -1.00e+00   6.7e-06 1.0e+00 
11 -1.00e+00   1.1e-07 1.0e+00 
sol = [1.0000000149131603, 1.5213299132729154e-9]
 k        fk ||∇f(x)||
 0  1.00e+00   4.5e+00
 1  3.33e-01   4.4e+00 3.0e-01 
 2 -4.00e-01   1.4e+00 1.0e+00 
 3 -6.13e-01   1.3e+00 1.0e+00 
 4 -9.01e-01   1.4e+00 6.7e-01 
 5 -9.82e-01   6.7e-01 1.0e+00 
 6 -9.98e-01   1.2e-01 1.0e+00 
 7 -9.99e-01   4.8e-02 1.0e+00 
 8 -1.00e+00   1.8e-03 1.0e+00 
 9 -1.00e+00   1.3e-04 1.0e+00 
10 -1.00e+00   9.0e-06 1.0e+00 
11 -1.00e+00   3.1e-09 1.0e+00 
sol = [0.9999999989781292, -5.078756032467174e-10]
Test Summary:                         | Pass  Total  Time
Test set for 

Test.DefaultTestSet("Test set for bfgs_quasi_newton_armijo", Any[], 5, false, false, true, 1.674495238018e9, 1.674495239514e9)

### Exercice 4: application à un problème de grande taille

On va ajouter le package `OptimizationProblems` qui contient, comme son nom l'indique, une collection de problème d'optimisation disponible au format de `JuMP` (dans le sous-module `OptimizationProblems.PureJuMP`) et de `ADNLPModel` (dans le sous-module `OptimizationProblems.ADNLPProblems`).

In [80]:
using Pkg;
Pkg.add("OptimizationProblems")

   Resolving package versions...


   Installed OptimizationProblems ─ v0.5.0


    Updating `C:\Users\Goglu\.julia\environments\v1.8\Project.toml`
  [5049e819] + OptimizationProblems v0.5.0
    Updating `C:\Users\Goglu\.julia\environments\v1.8\Manifest.toml`


  [6e4b80f9] + BenchmarkTools v1.3.2
  [523fee87] + CodecBzip2 v0.7.2
  [4076af6c] + JuMP v1.6.0
  [b8f27783] + MathOptInterface v1.11.4
  [d8a4904e] + MutableArithmetics v1.1.0
  [5049e819] + OptimizationProblems v0.5.0
  [9abbd945] + Profile


Precompiling 

project...


  ✓ OptimizationProblems


  1 dependency successfully precompiled in 92 seconds. 322 already precompiled.


In [81]:
using ADNLPModels, OptimizationProblems.ADNLPProblems # Attention si vous ne faites pas using ADNLPModels avant ça ne fonctionne pas!

In [82]:
n = 500
model = genrose(n=n)
@test typeof(model) <: ADNLPModel

Test Passed

Si vous le souhaitez, il est possible d'accéder à certaines informations sur le problème en accédant à son meta:

In [83]:
using OptimizationProblems
OptimizationProblems.genrose_meta

Dict{Symbol, Any} with 17 entries:
  :has_equalities_only    => false
  :origin                 => :unknown
  :has_inequalities_only  => false
  :defined_everywhere     => missing
  :has_fixed_variables    => false
  :variable_ncon          => false
  :nvar                   => 100
  :is_feasible            => true
  :minimize               => true
  :ncon                   => 0
  :name                   => "genrose"
  :best_known_lower_bound => -Inf
  :objtype                => :other
  :best_known_upper_bound => 405.106
  :has_bounds             => false
  :variable_nvar          => true
  :contype                => :unconstrained

Il est aussi possible d'accéder au meta de tous les problèmes

In [84]:
OptimizationProblems.meta

Row,nvar,variable_nvar,ncon,variable_ncon,minimize,name,has_equalities_only,has_inequalities_only,has_bounds,has_fixed_variables,objtype,contype,best_known_lower_bound,best_known_upper_bound,is_feasible,defined_everywhere,origin
,Int64,Bool,Int64,Bool,Bool,String,Bool,Bool,Bool,Bool,Symbol,Symbol,Real,Real,Bool?,Bool?,Symbol
1,1,false,0,false,true,AMPGO02,false,false,false,false,other,unconstrained,-Inf,0.839498,true,missing,unknown
2,1,false,0,false,true,AMPGO03,false,false,false,false,other,unconstrained,-Inf,2.88961,true,missing,unknown
3,1,false,0,false,true,AMPGO04,false,false,false,false,other,unconstrained,-Inf,-2.5666,true,missing,unknown
4,1,false,0,false,true,AMPGO05,false,false,false,false,other,unconstrained,-Inf,-0.0,true,missing,unknown
5,1,false,0,false,true,AMPGO06,false,false,false,false,other,unconstrained,-Inf,3.5177e-43,true,missing,unknown
6,1,false,0,false,true,AMPGO07,false,false,false,false,other,unconstrained,-Inf,2.56475,true,missing,unknown
7,1,false,0,false,true,AMPGO08,false,false,false,false,other,unconstrained,-Inf,-2.0928,true,missing,unknown
8,1,false,0,false,true,AMPGO09,false,false,false,false,other,unconstrained,-Inf,0.921136,true,missing,unknown
9,1,false,0,false,true,AMPGO10,false,false,false,false,other,unconstrained,-Inf,-0.0,true,missing,unknown


Résoudre le problème `genrose` et un autre problème de la collection en utilisant vos algorithmes précédents.
Avant d'utiliser l'algorithme on testera que le problème est bien sans contrainte avec:

In [85]:
unconstrained(nlp) #qui retourne vrai si `nlp` est un problème sans contraintes.


true

In [122]:
# Use previous functions to solve genrose.
x0 = ones(500)*0.99
reset!(model)
sol = bfgs_quasi_newton_armijo(model, x0, verbose=true) 

 k        fk ||∇f(x)||
 0  5.94e+00   4.4e+01
 1  2.58e+00   3.2e+01 7.7e-03 
 2  2.17e+00   5.5e+01 1.0e+00 


 3  1.50e+00   2.3e+01 4.4e-01 
 4  1.44e+00   3.0e+01 6.7e-01 
 5  1.22e+00   2.0e+01 6.7e-01 
 6  1.10e+00   1.2e+01 4.4e-01 
 7  1.08e+00   1.3e+01 6.7e-01 
 8  1.05e+00   9.6e+00 6.7e-01 
 9  1.04e+00   8.7e+00 6.7e-01 
10  1.03e+00   7.0e+00 6.7e-01 


11  1.02e+00   6.1e+00 6.7e-01 
12  1.01e+00   4.8e+00 6.7e-01 
13  1.01e+00   4.5e+00 6.7e-01 
14  1.01e+00   3.3e+00 6.7e-01 
15  1.01e+00   3.2e+00 6.7e-01 
16  1.00e+00   2.4e+00 6.7e-01 


17  1.00e+00   2.2e+00 6.7e-01 
18  1.00e+00   1.8e+00 6.7e-01 
19  1.00e+00   1.4e+00 6.7e-01 
20  1.00e+00   1.3e+00 6.7e-01 
21  1.00e+00   1.0e+00 6.7e-01 
22  1.00e+00   8.9e-01 6.7e-01 


23  1.00e+00   7.3e-01 6.7e-01 
24  1.00e+00   6.0e-01 6.7e-01 
25  1.00e+00   5.3e-01 6.7e-01 
26  1.00e+00   4.2e-01 6.7e-01 


...

70  1.00e+00   1.3e-02 6.7e-01 
71  1.00e+00   1.2e-02 6.7e-01 
72  1.00e+00   9.6e-03 6.7e-01 
73  1.00e+00   8.0e-03 6.7e-01 
74  1.00e+00   6.8e-03 6.7e-01 
75  1.00e+00   5.6e-03 6.7e-01 
76  1.00e+00   4.8e-03 6.7e-01 


77  1.00e+00   3.9e-03 6.7e-01 
78  1.00e+00   3.3e-03 6.7e-01 
79  1.00e+00   2.8e-03 6.7e-01 
80  1.00e+00   2.3e-03 6.7e-01 
81  1.00e+00   2.0e-03 6.7e-01 
82  1.00e+00   1.6e-03 6.7e-01 
83  1.00e+00   1.4e-03 6.7e-01 
84  1.00e+00   1.2e-03 6.7e-01 


85  1.00e+00   9.5e-04 6.7e-01 
86  1.00e+00   8.1e-04 6.7e-01 
87  1.00e+00   6.7e-04 6.7e-01 
88  1.00e+00   5.6e-04 6.7e-01 
89  1.00e+00   4.7e-04 6.7e-01 
90  1.00e+00   3.9e-04 6.7e-01 
91  1.00e+00   3.3e-04 6.7e-01 
92  1.00e+00   2.8e-04 6.7e-01 
93  1.00e+00   2.3e-04 6.7e-01 


94  1.00e+00   2.0e-04 6.7e-01 
95  1.00e+00   1.6e-04 6.7e-01 
96  1.00e+00   1.4e-04 6.7e-01 
97  1.00e+00   1.1e-04 6.7e-01 
98  1.00e+00   9.5e-05 6.7e-01 
99  1.00e+00   8.0e-05 6.7e-01 
100  1.00e+00   6.6e-05 6.7e-01 

METHOD FAILED - Reached Maximum Number of Iterations (100) 


500-element Vector{Float64}:
 0.9999999977190717
 0.9999999990799523
 0.9999999944858622
 0.9999999931392778
 0.9999999953346848
 0.9999999995751893
 1.0000000031136616
 1.0000000032505951
 1.000000000229861
 0.9999999974986471
 ⋮
 1.0000000023781461
 0.9999999989418505
 0.9999999950199809
 0.9999999936459192
 0.9999999955417725
 0.999999999639833
 1.000000003501212
 1.0000000037618333
 1.0000000111199292